# Importing Essential Modules

In [6]:
import librosa
import os
import numpy as np
import csv
import pandas as pd

# Extracting the necessary features for Baby Cry Audio

In [7]:
def cry_reason_cal(file_path):
  belly_pain_audio_path = "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/belly_pain"
  burping_audio_path =  "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/burping"
  discomfort_audio_path = "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/discomfort"
  hungry_audio_path = "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/hungry"
  tired_audio_path = "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/tired"

  if belly_pain_audio_path in file_path:
    return 0
  elif burping_audio_path in file_path:
    return 1
  elif discomfort_audio_path in file_path:
    return 2
  elif hungry_audio_path in file_path:
    return 3
  elif tired_audio_path in file_path:
    return 4
  else :
    return -1

In [8]:
#fuction for feature extraction

def feature_extraction(file_path):
  # loading of the audio file
  audio, sample_rate = librosa.load(file_path)

  # extracting mfcc feature
  mfcc = np.mean(librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 13).T, axis = 0)

  # extracting amplitude_envelope_mean feature
  amplitude_envelope = np.abs(librosa.amplitude_to_db(librosa.stft(audio), ref=np.max))
  amplitude_envelope_mean = np.mean(amplitude_envelope)

  # extracting RMS_Mean feature
  RMS = librosa.feature.rms(y=audio)[0]
  RMS_Mean = np.mean(RMS)

  # extracting ZCR_Mean feature
  ZCR = librosa.feature.zero_crossing_rate(y=audio)[0]
  ZCR_Mean = np.mean(ZCR)


  # extracting STFT_Mean feature
  STFT_Mean = np.abs(librosa.stft(audio))
  STFT_Mean = np.mean(STFT_Mean)

  # extracting SC_Mean feature
  spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)[0]
  SC_Mean = np.mean(spectral_centroids)

  # Cry Reason representation by numbers
  # 0 - belly pain
  # 1 - burping
  # 2 - discomfort
  # 3 - hungry
  # 4 - tired



  return [amplitude_envelope_mean, RMS_Mean, ZCR_Mean, STFT_Mean, SC_Mean, mfcc, cry_reason_cal(file_path)]


In [9]:
mfcc_features = {}
directory = "/content/drive/MyDrive/Baby Cry ML/donateacry_corpus_cleaned_and_updated_data/"
for file in os.listdir(directory):
  if os.path.isdir(os.path.join(directory, file)):
    for audio in os.listdir(os.path.join(directory, file) + "/"):
      mfcc_features[audio] = feature_extraction(os.path.join(directory, file) + "/" + audio)

<ipython-input-8-482315de4694>:11: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  amplitude_envelope = np.abs(librosa.amplitude_to_db(librosa.stft(audio), ref=np.max))


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
def save_to_csv(dictionary, csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Audio File', 'amplitude_envelope_mean', 'RMS_Mean', 'ZCR_Mean', 'STFT_Mean', 'SC_Mean', 'MFCCs1','MFCCs2','MFCCs3','MFCCs4','MFCCs5','MFCCs6','MFCCs7','MFCCs8','MFCCs9','MFCCs10','MFCCs11','MFCCs12','MFCCs13', 'Cry_Reason']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for audio_file, features in dictionary.items():
            writer.writerow({'Audio File': audio_file, 'amplitude_envelope_mean': features[0], 'RMS_Mean': features[1], 'ZCR_Mean': features[2], 'STFT_Mean': features[3], 'SC_Mean': features[4], 'MFCCs1': features[5][0],'MFCCs2': features[5][1],'MFCCs3': features[5][2],'MFCCs4': features[5][3],'MFCCs5': features[5][4],'MFCCs6': features[5][5],'MFCCs7': features[5][6],'MFCCs8': features[5][7],'MFCCs9': features[5][8],'MFCCs10': features[5][9],'MFCCs11': features[5][10],'MFCCs12': features[5][11],'MFCCs13': features[5][12], 'Cry_Reason': features[6]})

In [12]:
save_to_csv(mfcc_features, 'mfcc_features.csv')

df = pd.read_csv('mfcc_features.csv')

In [13]:
df

,Audio File,amplitude_envelope_mean,RMS_Mean,ZCR_Mean,STFT_Mean,SC_Mean,MFCCs1,MFCCs2,MFCCs3,MFCCs4,MFCCs5,MFCCs6,MFCCs7,MFCCs8,MFCCs9,MFCCs10,MFCCs11,MFCCs12,MFCCs13,Cry_Reason
0,04c3386b-e6bc-4bd0-8456-d46ae21a73fc-143530582...,72.815506,0.004536,0.095788,0.021560,1314.144617,-604.95470,137.22401,-51.586520,-11.703494,5.824594,-28.804176,-1.659406,2.087006,-11.208941,6.900043,-2.930204,-12.498389,4.121874,3
1,090C15A8-5406-4EA5-97A3-81F6527227C0-143014751...,71.364540,0.048529,0.122662,0.353793,1544.687853,-341.23920,125.25603,-73.734400,-4.122528,21.787447,-25.596094,-5.019836,-9.761853,-21.945234,5.851743,-7.101247,-17.897327,6.027918,3
2,02c3b725-26e4-4a2c-9336-04ddc58836d9-143072619...,70.098854,0.000744,0.115793,0.005222,1407.833053,-706.70026,160.40320,-59.174213,11.621321,16.054573,-40.199245,6.834778,8.181551,-19.020689,7.330266,-3.399242,-14.793996,9.604260,3
3,090C15A8-5406-4EA5-97A3-81F6527227C0-143014752...,75.233660,0.059639,0.112877,0.397042,1433.009805,-342.74120,96.51876,-43.875446,-12.724279,13.519671,-10.782829,-8.929214,-13.451996,-13.951735,4.089157,-6.331817,-14.008081,2.926110,3
4,20E2B5A0-40A4-4FD8-8935-AFD1DB833510-143012767...,70.545970,0.040463,0.063135,0.250313,1060.123157,-338.35535,168.32964,-48.833668,13.194779,33.635826,-19.791740,7.405217,9.822707,-13.391487,3.092201,-7.924339,-19.766375,-2.817857,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,06c4cfa2-7fa6-4fda-91a1-ea186a4acc64-143002924...,73.768490,0.024638,0.093969,0.160996,1270.155480,-433.07892,124.73902,-41.795440,-1.283541,18.879490,-22.776611,-5.581900,3.240490,-15.555840,-5.476026,-6.403492,-19.308160,-9.878424,4
336,B2AABCE0-1F62-4B57-B58A-308959ABDFE7-143718875...,72.775070,0.068853,0.106073,0.334607,1336.469630,-332.34800,151.82790,-66.246200,2.983666,26.656265,-33.877613,-6.559297,5.100830,-18.202543,-4.027010,-7.077313,-15.265632,-2.142721,4
337,1309B82C-F146-46F0-A723-45345AFA6EA8-143005986...,67.491425,0.021048,0.111789,0.185632,1570.755424,-373.48320,174.15509,-89.780655,13.069295,23.626520,-43.226980,16.783014,10.220210,-24.853378,15.905594,-2.321229,-26.592060,6.575035,4
338,7A22229D-06C2-4AAA-9674-DE5DF1906B3A-143689195...,72.085850,0.051533,0.106426,0.300572,1478.556537,-335.60336,136.83118,-64.568150,6.508425,12.319852,-41.034653,-1.479996,-6.934961,-30.265340,3.118389,-6.849393,-21.497957,4.431449,4
